In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier, ExtraTreesClassifier
from sklearn.datasets import load_wine

from palmerpenguins import load_penguins

In [ ]:
penguins_raw = load_penguins()
penguins = (
    penguins_raw
    .dropna()
    .drop(columns=["island", "sex", "year"])
)

In [ ]:
penguins.head()

In [ ]:
sns.pairplot(penguins, hue="species", height=2.5)

In [ ]:
ax = sns.jointplot(
    data=penguins,
    x="bill_length_mm",
    y="flipper_length_mm",
    hue="species"
)

In [ ]:
ax = sns.jointplot(
    data=penguins,
    x="bill_length_mm",
    y="flipper_length_mm",
    hue="species"
)
ax.ax_joint.axhline(206, c='r');

In [ ]:
ax = sns.jointplot(
    data=penguins.loc[lambda x: x["flipper_length_mm"] < 206],
    x="bill_length_mm",
    y="flipper_length_mm",
    hue="species"
)
ax.ax_joint.axvline(44, c='r');

## Trying out some Decision Trees

In [ ]:
cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

In [ ]:
X = penguins.drop(columns="species")
y = penguins["species"]

In [ ]:
y.value_counts()

In [ ]:
y.value_counts() / y.value_counts().sum()

## Fitting the models

In [ ]:
tree1 = DecisionTreeClassifier(max_depth=1)

In [ ]:
tree1_scores = cross_val_score(tree1, X, y, scoring="accuracy", cv=cv)

In [ ]:
tree1_scores

In [ ]:
tree1_scores.mean()

In [ ]:
tree2 = DecisionTreeClassifier(max_depth=2)

In [ ]:
tree2_scores = cross_val_score(tree2, X, y, scoring="accuracy", cv=cv)

In [ ]:
tree2_scores

In [ ]:
tree2_scores.mean()

## Plotting the Trees

In order to plot them using this `plot_tree` function, the trees must first be fitted. So here I am fitting them to the whole training set, even though this would not be used when assessing the performance of the model

In [ ]:
tree1.fit(X, y)

In [ ]:
plot_tree(tree1);

In [ ]:
np.unique(tree1.predict(X))

In [ ]:
X.columns

In [ ]:
tree2.fit(X, y)

In [ ]:
plot_tree(tree2);

In [ ]:
ax = sns.jointplot(
    data=penguins,
    x="bill_depth_mm",
    y="flipper_length_mm",
    hue="species"
)

ax.ax_joint.axvline(17.65, c='r')
ax.ax_joint.axhline(206.5, c='r');

## Now looking at the wine dataset

In [ ]:
wine_raw = load_wine()

In [ ]:
wine_raw.keys()

In [ ]:
wine_df = pd.DataFrame(
    data=wine_raw["data"],
    columns=wine_raw["feature_names"]
)

full_wine_df = (
    wine_df
    .assign(**{
        "Type": wine_raw["target"]
    })
)

In [ ]:
full_wine_df["Type"].value_counts()

In [ ]:
wine_df.corr()

In [ ]:
sns.clustermap(
    data=wine_df.corr(method="spearman"),
    method="ward",
    robust=True
)

## Fitting the tree

In [ ]:
wine_X = wine_df.copy()
wine_y = full_wine_df["Type"]

In [ ]:
wine_tree = DecisionTreeClassifier(max_depth=2)

In [ ]:
cross_val_score(wine_tree, wine_X, wine_y, cv=cv)

In [ ]:
wine_bagging = BaggingClassifier(
    base_estimator=wine_tree,
    n_estimators=10,
    bootstrap=True,
    bootstrap_features=True
)

In [ ]:
cross_val_score(wine_bagging, wine_X, wine_y, cv=cv)

In [ ]:
rf_wine = RandomForestClassifier(
    n_estimators=10
)

In [ ]:
cross_val_score(rf_wine, wine_X, wine_y, cv=cv)

In [ ]:
et_wine = ExtraTreesClassifier(
    n_estimators=100
)

In [ ]:
cross_val_score(et_wine, wine_X, wine_y, cv=cv)